In [1]:
month_id = 202212

## Daily Aggregrage
- dictionary is done
- aggreage both domain, tag and tag_gp

### 0. Initialization

In [2]:
import cx_Oracle
import pandas as pd
from smtutil import common
from smtutil import hiveutil
from datetime import datetime, timedelta
import numpy as np
import csv, pickle
import calendar

Connect to Oracle Successfully
Connect to Oracle Successfully


In [3]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [4]:
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)

In [5]:
month_id = int((datetime.now().replace(day=1) - timedelta(days=1)).strftime("%Y%m"))
date_id_st = month_id * 100 + 1
date_id_end = month_id * 100 + calendar.monthrange(month_id // 100, month_id % 100)[1]
days = date_id_end - date_id_st + 1
print(date_id_st, date_id_end, days)

20221201 20221231 31


In [6]:
spark.sql("refresh table weblog.t_domain_monthly")
spark.sql("refresh table weblog.t_tag_monthly")
spark.sql("refresh table weblog.t_tag_gp_monthly")

DataFrame[]

In [7]:
count_days = spark.sql("""
select count(distinct(date_id)) as count_days
from weblog.t_domain_daily
where date_id between {date_id_st} and {date_id_end}
""".format(
    date_id_st=date_id_st,
    date_id_end=date_id_end
)).first()['count_days']
is_data_ready = (count_days == days)
is_data_ready

KeyboardInterrupt: 

In [ ]:
is_loaded = spark.sql("""
select 1
from weblog.t_domain_monthly
where month_id = {month_id}
limit 1
""".format(
    month_id=month_id
)).count()
is_loaded

In [ ]:
is_to_process = is_data_ready and (is_loaded == 0)
is_to_process

In [ ]:
spark.sql("""
select distinct
    month_id
from weblog.t_domain_monthly
order by month_id
""").toPandas()

### 1. Aggregation

#### 1.1. Aggregate Domain

In [ ]:
if is_to_process:
    print(datetime.now()) # 2 hrs
    spark.sql(f"alter table weblog.t_domain_monthly drop if exists partition (month_id={month_id})")
    spark.sql("""
    select
        subr_num,
        agg_value as agg_value,
        agg_level as agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        sum(duration) as duration,
        count(distinct(date_id)) as days,
        {month_id} as month_id
    from weblog.t_domain_daily
    where date_id between {date_id_st} and {date_id_end}
    group by subr_num, agg_value, agg_level
    """.format(
        month_id=month_id,
        date_id_st=date_id_st,
        date_id_end=date_id_end
    ))\
        .write.mode('append')\
        .partitionBy('month_id')\
        .saveAsTable('weblog.t_domain_monthly')
    print(datetime.now())

In [ ]:
%%javascript
alert('done')

### 1.2. Aggregate Tag

In [ ]:
date_id_st, date_id_end

In [ ]:
if is_to_process:
    exclude_tag_agg_values = spark.sql("""
    select distinct
        t_tag_dictionary.agg_value
    from  weblog.t_tag_dictionary
    where default_aggregation = 0
    """).toPandas().agg_value
    exclude_tag_agg_values = list(exclude_tag_agg_values)
    exclude_tag_agg_values_str = ','.join([str(_) for _ in exclude_tag_agg_values])

In [ ]:
if is_to_process:
    print(datetime.now()) # 5 min
    spark.sql(f"alter table weblog.t_tag_monthly drop if exists partition (month_id={month_id})")
    spark.sql("""
    select
        subr_num,
        agg_value as agg_value,
        agg_level as agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        sum(duration) as duration,
        count(distinct(date_id)) as days,
        {month_id} as month_id
    from weblog.t_tag_daily
    where date_id between {date_id_st} and {date_id_end}
    and agg_value not in ({exclude_tag_agg_values_str})
    group by subr_num, agg_value, agg_level
    """.format(
        month_id=month_id,
        date_id_st=date_id_st,
        date_id_end=date_id_end,
        exclude_tag_agg_values_str=exclude_tag_agg_values_str
    ))\
        .write.mode('append')\
        .partitionBy('month_id')\
        .saveAsTable('weblog.t_tag_monthly')
    print(datetime.now())

In [ ]:
%%javascript
alert('done')

In [ ]:
if is_to_process:
    spark.sql("""
    select *
    from weblog.t_tag_daily
    where date_id between {date_id_st} and {date_id_end}
    limit 3
    """.format(
        month_id=month_id,
        date_id_st=date_id_st,
        date_id_end=date_id_end
    )).toPandas()

### 1.3. Aggregate Tag Group

In [ ]:
if is_to_process:
    exclude_tag_gp_agg_values = spark.sql("""
    select distinct
        agg_value
    from  weblog.t_tag_gp_dictionary
    where default_aggregation = 0
    """).toPandas().agg_value
    exclude_tag_agg_values_str = ','.join([str(_) for _ in exclude_tag_agg_values])

In [ ]:
if is_to_process:
    print(datetime.now()) # 30 sec
    spark.sql(f"alter table weblog.t_tag_gp_monthly drop if exists partition (month_id={month_id})")
    spark.sql("""
    select
        subr_num,
        agg_value as agg_value,
        agg_level as agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        sum(duration) as duration,
        count(distinct(date_id)) as days,
        {month_id} as month_id
    from weblog.t_tag_gp_daily
    where date_id between {date_id_st} and {date_id_end}
    and agg_value not in ({exclude_tag_agg_values_str})
    group by subr_num, agg_value, agg_level
    """.format(
        month_id=month_id,
        date_id_st=date_id_st,
        date_id_end=date_id_end,
        exclude_tag_agg_values_str=exclude_tag_agg_values_str
    ))\
        .write.mode('append')\
        .partitionBy('month_id')\
        .saveAsTable('weblog.t_tag_gp_monthly')
    print(datetime.now())

In [ ]:
%%javascript
alert('done')